# Data Merging
**The purpose of this notebook is to merge data found from the AIMS eReefs Visualisation Portal with the Reef Check/CoRTAD data that we were introduced to during the labs. Data from AIMS is in NetCDF format, which have them been converted to CSV, using "NetCDF-conversion.py". Merging method is by coordinates of data points (longitude and latitude). Exact matching of data was not achievable, as the coordinates won't match. The decision was to allow a half degree difference when matching, in each axis.**

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Data
data_dir = "./Data/"
reef_check_path = data_dir + "Reef_Check_with_cortad_variables_with_annual_rate_of_SST_change.csv"
aims_path = data_dir + "eReefs-aggregated-monthly-data.csv"

## Inspecting Data

In [3]:
# Read data and have a brief look
reef_check = pd.read_csv(reef_check_path)
aims = pd.read_csv(aims_path)

# Conver date in reef check to datetime dtype
reef_check['Date'] = pd.to_datetime(reef_check['Date'])

# Add month column
reef_check['Month'] = reef_check['Date'].dt.month

# Add index column
reef_check['index'] = range(1, len(reef_check)+1)

In [4]:
reef_check.head()

,Reef.ID,Reef.Name,Ocean,Country,State.Province.Island,City.Town,Year,Date,Depth,Organism.Code,...,TSA_DHW_Standard_Deviation,TSA_DHWMax,TSA_DHWMean,Region,Diversity,rate_of_SST_change,Longitude.Degrees,Latitude.Degrees,Month,index
0,103.10.28.1E.10.50.46.1N,Koh Mano (Minor),Indian,Cambodia,Koh Kong,NaN,2003,2003-02-19,4.5,Bleaching (% of population),...,1.690,14.560,0.590,ERG050,417.0,0.020556,103.174472,10.846139,2,1
1,103.11.35.5E.10.49.32N,Koh Mano (south),Indian,Cambodia,Koh Kong,NaN,2003,2003-02-28,4.5,Bleaching (% of population),...,1.800,17.010,0.520,ERG050,417.0,0.020556,103.193194,10.825556,2,2
2,103.11.79.5E.10.48.2.7N,Koh Ta Team,Indian,Cambodia,Koh Kong,NaN,2003,2003-02-24,5.0,Bleaching (% of population),...,1.620,13.760,0.520,ERG050,417.0,0.020556,103.196583,10.800750,2,3
3,103.4.16.8E.11.3.36.2N,Koh Krosa Krao,Indian,Cambodia,Koh Kong,NaN,2003,2003-02-25,6.0,Bleaching (% of population),...,1.450,10.940,0.410,ERG050,417.0,0.019066,103.071333,11.060056,2,4
4,103.4.63.9E.11.3.58.3N,Koh Krosa Kandal,Indian,Cambodia,Koh Kong,NaN,2003,2003-02-27,3.2,Bleaching (% of population),...,1.692,15.038,0.488,ERG050,417.0,0.019066,103.077306,11.066194,2,5


In [5]:
aims.head()

,latitude,longitude,eta,salt,temp,wspeed_u,wspeed_v,mean_wspeed,u,v,mean_cur,year,month
0,-28.696022,153.688788,-0.305821,35.540714,21.575321,0.235666,-1.500159,6.352422,0.062902,-0.641632,0.793684,2010,9
1,-28.696022,153.808788,-0.247241,35.518906,22.374939,0.354274,-1.771560,6.826605,0.001102,-0.895112,0.946391,2010,9
2,-28.696022,153.928788,-0.171045,35.508064,22.678856,0.277497,-1.897144,6.999744,-0.055618,-0.952047,0.978366,2010,9
3,-28.696022,154.048788,-0.099933,35.533850,22.471252,0.164686,-1.925546,7.053525,-0.033786,-0.801583,0.813448,2010,9
4,-28.696022,154.168788,-0.042675,35.561455,22.296648,0.073256,-1.877180,7.077173,-0.008968,-0.542457,0.563871,2010,9


**Each of the data sets have columns corresponding to longitude and latitude. All data points in AIMS are in Australia, particularly the Great Barrier Reef, so all are valid for us. It would be useful to know how many data points in the Reef Check data set are located in this area. According to Marineregions.org, the bounding box for the Great Barrier Reef is Min. Lat	22° 26' 35.7" S (-22.4433°), Min. Long	142° 55' 48.1" E (142.93°), Max. Lat	9° 13' 59.3" S (-9.2331°), Max. Long	152° 49' 25.6" E (152.8238°). However, the data in AIMS suggests otherwise. The bounding box used for filtering GBR data points in the reef check data should match that of AIMS'. A half degree leeway is added to each axis in both direction.**

Marineregions.org webpage for GBR: https://www.marineregions.org/gazetteer.php?p=details&id=7579 

In [6]:
print("Longitude and latitude range of data in AIMS")
print(min(aims['latitude']), max(aims['latitude']))
print(min(aims['longitude']), max(aims['longitude']))

Longitude and latitude range of data in AIMS
-28.696022 -7.576022
142.408788 156.808788


In [7]:
reef_check.dtypes

Reef.ID                                          object
Reef.Name                                        object
Ocean                                            object
Country                                          object
State.Province.Island                            object
City.Town                                        object
Year                                              int64
Date                                     datetime64[ns]
Depth                                           float64
Organism.Code                                    object
S1                                              float64
S2                                              float64
S3                                              float64
S4                                              float64
Errors.                                            bool
What.errors.                                     object
Average_bleaching                               float64
ClimSST                                         

In [8]:
# Count number of Great Barrier Reef data points in Reef Check
gbr_reefcheck = reef_check[(reef_check['Longitude.Degrees'] <= max(aims['longitude'])+0.5) &\
                           (reef_check['Longitude.Degrees'] >= min(aims['longitude'])-0.5) &\
                           (reef_check['Latitude.Degrees'] >= min(aims['latitude'])-0.5) &\
                           (reef_check['Latitude.Degrees'] <= max(aims['latitude'])+0.5)
                          ]


In [9]:
print("Reef check dimension:", reef_check.shape)
print("Great Barrier Reef points in reef check: ", gbr_reefcheck.shape)
print("AIMS dimension: ", aims.shape)

Reef check dimension: (9215, 57)
Great Barrier Reef points in reef check:  (642, 57)
AIMS dimension:  (799656, 13)


## Exact-match-merging doesn't work (as expected)

In [10]:
# Trying exact merge, as shown below, it doesn't work
gbr_reefcheck = gbr_reefcheck.rename(columns = {'Longitude.Degrees':'reef_longitude', 'Latitude.Degrees':'reef_latitude'})
exact_merge = pd.merge(gbr_reefcheck, aims, how="inner", left_on=["reef_latitude", "reef_longitude"], right_on=['latitude', 'longitude'])
exact_merge.shape

(0, 70)

In [11]:
print(exact_merge.columns)

Index(['Reef.ID', 'Reef.Name', 'Ocean', 'Country', 'State.Province.Island',
       'City.Town', 'Year', 'Date', 'Depth', 'Organism.Code', 'S1', 'S2', 'S3',
       'S4', 'Errors.', 'What.errors.', 'Average_bleaching', 'ClimSST',
       'Temperature_Kelvin', 'Temperature_Mean', 'Temperature_Minimum',
       'Temperature_Maximum', 'Temperature_Kelvin_Standard_Deviation',
       'Windspeed', 'SSTA', 'SSTA_Standard_Deviation', 'SSTA_Mean',
       'SSTA_Minimum', 'SSTA_Maximum', 'SSTA_Frequency',
       'SSTA_Frequency_Standard_Deviation', 'SSTA_FrequencyMax',
       'SSTA_FrequencyMean', 'SSTA_DHW', 'SSTA_DHW_Standard_Deviation',
       'SSTA_DHWMax', 'SSTA_DHWMean', 'TSA', 'TSA_Standard_Deviation',
       'TSA_Minimum', 'TSA_Maximum', 'TSA_Mean', 'TSA_Frequency',
       'TSA_Frequency_Standard_Deviation', 'TSA_FrequencyMax',
       'TSA_FrequencyMean', 'TSA_DHW', 'TSA_DHW_Standard_Deviation',
       'TSA_DHWMax', 'TSA_DHWMean', 'Region', 'Diversity',
       'rate_of_SST_change', 'reef_long

## Merging with leeway for coordinates
**As mentioned before, leeway of 0.5 degrees in each direction is given, to help with merging. Method used was adapted from the stackoverflow answer by piRSquared, linked below.**

https://stackoverflow.com/questions/44367672/best-way-to-join-merge-by-range-in-pandas

In [12]:
# Creating columns for indicating coordinates range in AIMS
aims['low_lat'] = aims['latitude'] - 0.5
aims['high_lat'] = aims['latitude'] + 0.5
aims['low_long'] = aims['longitude'] - 0.5
aims['high_long'] = aims['longitude'] + 0.5

In [13]:
reefcheck_lat = gbr_reefcheck.reef_latitude.values
reefcheck_long = gbr_reefcheck.reef_longitude.values
low_lat = aims.low_lat.values
high_lat = aims.high_lat.values
low_long = aims.low_long.values
high_long = aims.high_long.values
reefcheck_month = gbr_reefcheck.Month.values
reefcheck_year = gbr_reefcheck.Year.values 
aims_month = aims.month.values
aims_year = aims.year.values

# Find index of rows where reefcheck coordinates within range (comparing month and year causes an elemenwise comparison failure)
i, j = np.where( ((reefcheck_lat[:, None] >= low_lat) & (reefcheck_lat[:, None] <= high_lat)) &\
                 ((reefcheck_long[:, None] >= low_long) & (reefcheck_long[:, None] <= high_long)) 
                 # ((reefcheck_month == aims_month) & (reefcheck_year == aims_year))
               )

temp_merged = pd.concat([gbr_reefcheck.iloc[i, :].reset_index(drop=True),
          aims.iloc[j, :].reset_index(drop=True)]
    , axis=1)

In [14]:
# Note this is with invalid rows, as month and year has not been matched yet
temp_merged.shape

(3145472, 74)

In [15]:
# Matching month and year from both data sets. Capitalized cols are from reefcheck.
merged = temp_merged[(temp_merged.Month==temp_merged.month) & (temp_merged.Year==temp_merged.year)]
merged.shape

(14096, 74)

In [16]:
merged.head()

,Reef.ID,Reef.Name,Ocean,Country,State.Province.Island,City.Town,Year,Date,Depth,Organism.Code,...,mean_wspeed,u,v,mean_cur,year,month,low_lat,high_lat,low_long,high_long
1337658,153.29.29E.26.58.7S,Flinders Reef - Aladdin's Cave (Fringing reef ...,Pacific,Australia,Queensland,NaN,2010,2010-10-17,10.0,Bleaching (% of population),...,5.232096,0.030541,0.005361,0.185999,2010,10,-27.876022,-26.876022,152.708788,153.708788
1337659,153.29.29E.26.58.7S,Flinders Reef - Aladdin's Cave (Fringing reef ...,Pacific,Australia,Queensland,NaN,2010,2010-10-17,10.0,Bleaching (% of population),...,6.145797,-0.047292,0.002193,0.279372,2010,10,-27.876022,-26.876022,152.828788,153.828788
1337660,153.29.29E.26.58.7S,Flinders Reef - Aladdin's Cave (Fringing reef ...,Pacific,Australia,Queensland,NaN,2010,2010-10-17,10.0,Bleaching (% of population),...,6.112157,0.037247,0.001953,0.520450,2010,10,-27.876022,-26.876022,152.948788,153.948788
1337661,153.29.29E.26.58.7S,Flinders Reef - Aladdin's Cave (Fringing reef ...,Pacific,Australia,Queensland,NaN,2010,2010-10-17,10.0,Bleaching (% of population),...,6.153152,0.023441,-0.234490,0.362635,2010,10,-27.876022,-26.876022,153.068788,154.068788
1337662,153.29.29E.26.58.7S,Flinders Reef - Aladdin's Cave (Fringing reef ...,Pacific,Australia,Queensland,NaN,2010,2010-10-17,10.0,Bleaching (% of population),...,6.168165,-0.014926,-0.402495,0.462817,2010,10,-27.876022,-26.876022,153.188788,154.188788


In [17]:
print(merged.Ocean.unique(), merged.Country.unique(), merged.year.unique())

['Pacific'] ['Australia'] [2010 2011 2012 2013 2015 2014 2016]


In [18]:
print(min(merged['Year']), max(merged['Year']))

2010 2016


In [19]:
# Save this version of data as machine learning csv
merged.to_csv("./Data/ml_version_merged.csv")

## Group data
**Each reef check data point can be matched to multiple AIMS data points (1:N relationship), ‘cause I’m only matching based on the range in the values of longitude and latitude (not matching reef check to nearest data point in AIMS, which would be 1:1). This, though would be fine for machine learning models, would pose a problem for regression models. Hence, for each of the reef points, the data from AIMS is averaged.**

In [20]:
# Remove redundant columns
merged = merged.drop(['Depth', 'Organism.Code', 'Date', 'low_lat', 'high_lat', 'low_long', 'high_long', 'year', 'month', 'longitude', 'latitude', 'Errors.', 'What.errors.', 'State.Province.Island', 'City.Town', 'Region', 'Diversity'], axis=1)
merged.shape

(14096, 57)

In [21]:
# Columns in merged data
merged.columns

Index(['Reef.ID', 'Reef.Name', 'Ocean', 'Country', 'Year', 'S1', 'S2', 'S3',
       'S4', 'Average_bleaching', 'ClimSST', 'Temperature_Kelvin',
       'Temperature_Mean', 'Temperature_Minimum', 'Temperature_Maximum',
       'Temperature_Kelvin_Standard_Deviation', 'Windspeed', 'SSTA',
       'SSTA_Standard_Deviation', 'SSTA_Mean', 'SSTA_Minimum', 'SSTA_Maximum',
       'SSTA_Frequency', 'SSTA_Frequency_Standard_Deviation',
       'SSTA_FrequencyMax', 'SSTA_FrequencyMean', 'SSTA_DHW',
       'SSTA_DHW_Standard_Deviation', 'SSTA_DHWMax', 'SSTA_DHWMean', 'TSA',
       'TSA_Standard_Deviation', 'TSA_Minimum', 'TSA_Maximum', 'TSA_Mean',
       'TSA_Frequency', 'TSA_Frequency_Standard_Deviation', 'TSA_FrequencyMax',
       'TSA_FrequencyMean', 'TSA_DHW', 'TSA_DHW_Standard_Deviation',
       'TSA_DHWMax', 'TSA_DHWMean', 'rate_of_SST_change', 'reef_longitude',
       'reef_latitude', 'Month', 'index', 'eta', 'salt', 'temp', 'wspeed_u',
       'wspeed_v', 'mean_wspeed', 'u', 'v', 'mean_cur'],
 

In [22]:
# Group on all reef check data columns, calculate average for AIMS columns.
# reefcheck_cols = [col for col in merged.columns]
# remove = ['eta',
#        'salt', 'temp', 'wspeed_u', 'wspeed_v', 'mean_wspeed', 'u', 'v',
#        'mean_cur']
# for each in remove:
#     reefcheck_cols.remove(each)
# print(reefcheck_cols, len(reefcheck_cols))

# merged_grouped = merged.groupby(reefcheck_cols, dropna=False)

merged_grouped = merged.groupby(['index', 'Year', 'Month', 'reef_longitude', 'reef_latitude'], dropna=False)

In [23]:
print("We would have", merged_grouped.ngroups, "usable data points.")

We would have 271 usable data points.


In [24]:
merged_grouped.head(10)

,Reef.ID,Reef.Name,Ocean,Country,Year,S1,S2,S3,S4,Average_bleaching,...,index,eta,salt,temp,wspeed_u,wspeed_v,mean_wspeed,u,v,mean_cur
1337658,153.29.29E.26.58.7S,Flinders Reef - Aladdin's Cave (Fringing reef ...,Pacific,Australia,2010,0.0,0.0,0.0,0.0,0.0,...,5097,-0.342685,30.397068,22.630415,-2.501203,0.992031,5.232096,0.030541,0.005361,0.185999
1337659,153.29.29E.26.58.7S,Flinders Reef - Aladdin's Cave (Fringing reef ...,Pacific,Australia,2010,0.0,0.0,0.0,0.0,0.0,...,5097,-0.355578,34.308918,22.464676,-3.136570,1.240022,6.145797,-0.047292,0.002193,0.279372
1337660,153.29.29E.26.58.7S,Flinders Reef - Aladdin's Cave (Fringing reef ...,Pacific,Australia,2010,0.0,0.0,0.0,0.0,0.0,...,5097,-0.367143,34.895367,22.612470,-3.150469,1.158783,6.112157,0.037247,0.001953,0.520450
1337661,153.29.29E.26.58.7S,Flinders Reef - Aladdin's Cave (Fringing reef ...,Pacific,Australia,2010,0.0,0.0,0.0,0.0,0.0,...,5097,-0.364988,35.403656,23.069603,-3.224512,1.168771,6.153152,0.023441,-0.234490,0.362635
1337662,153.29.29E.26.58.7S,Flinders Reef - Aladdin's Cave (Fringing reef ...,Pacific,Australia,2010,0.0,0.0,0.0,0.0,0.0,...,5097,-0.343447,35.423850,23.448221,-3.270661,1.162936,6.168165,-0.014926,-0.402495,0.462817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3144289,153.409936E.27.472029S,Myora Reef - Myora Reef (Reef flat) Site 2,Pacific,Australia,2015,0.0,0.0,0.0,0.0,0.0,...,8638,-0.334937,35.606550,23.205894,-2.091114,-0.049086,5.690556,-0.021762,-0.132043,0.242538
3144290,153.409936E.27.472029S,Myora Reef - Myora Reef (Reef flat) Site 2,Pacific,Australia,2015,0.0,0.0,0.0,0.0,0.0,...,8638,-0.310877,35.607998,23.349714,-2.211005,-0.242468,5.697269,0.002152,-0.429296,0.455284
3144291,153.409936E.27.472029S,Myora Reef - Myora Reef (Reef flat) Site 2,Pacific,Australia,2015,0.0,0.0,0.0,0.0,0.0,...,8638,-0.284872,35.591114,23.459766,-2.303411,-0.402155,5.670225,0.027804,-0.405915,0.437426
3144292,153.409936E.27.472029S,Myora Reef - Myora Reef (Reef flat) Site 2,Pacific,Australia,2015,0.0,0.0,0.0,0.0,0.0,...,8638,-0.313424,35.887200,24.043785,-1.931378,0.274081,5.186780,0.000429,-0.000116,0.075048


In [25]:
# Don't average over year month coordinates
mean_merged_grouped_df = merged_grouped.mean()

In [26]:
mean_merged_grouped_df.columns

Index(['S1', 'S2', 'S3', 'S4', 'Average_bleaching', 'ClimSST',
       'Temperature_Kelvin', 'Temperature_Mean', 'Temperature_Minimum',
       'Temperature_Maximum', 'Temperature_Kelvin_Standard_Deviation',
       'Windspeed', 'SSTA', 'SSTA_Standard_Deviation', 'SSTA_Mean',
       'SSTA_Minimum', 'SSTA_Maximum', 'SSTA_Frequency',
       'SSTA_Frequency_Standard_Deviation', 'SSTA_FrequencyMax',
       'SSTA_FrequencyMean', 'SSTA_DHW', 'SSTA_DHW_Standard_Deviation',
       'SSTA_DHWMax', 'SSTA_DHWMean', 'TSA', 'TSA_Standard_Deviation',
       'TSA_Minimum', 'TSA_Maximum', 'TSA_Mean', 'TSA_Frequency',
       'TSA_Frequency_Standard_Deviation', 'TSA_FrequencyMax',
       'TSA_FrequencyMean', 'TSA_DHW', 'TSA_DHW_Standard_Deviation',
       'TSA_DHWMax', 'TSA_DHWMean', 'rate_of_SST_change', 'eta', 'salt',
       'temp', 'wspeed_u', 'wspeed_v', 'mean_wspeed', 'u', 'v', 'mean_cur'],
      dtype='object')

In [27]:
mean_merged_grouped_df['salt']

index  Year  Month  reef_longitude  reef_latitude
5097   2010  10     153.491389      -26.968611       35.194298
5102   2010  10     153.483333      -26.968333       35.194298
5106   2010  11     153.385556      -27.529167       35.172879
5132   2010  9      153.151667      -26.635000       35.578607
5134   2010  11     153.103056      -26.359722       35.449339
                                                       ...    
8630   2016  3      153.437333      -27.400694       35.385982
8631   2015  12     153.425611      -27.936889       35.500669
8636   2015  11     153.475833      -28.109167       35.547334
8637   2014  6      153.375250      -27.500000       35.396160
8638   2015  10     153.409944      -27.472028       35.650009
Name: salt, Length: 271, dtype: float64

## Save data

In [28]:
mean_merged_grouped_df.to_csv('./Data/lr_version_merged_mean.csv')